In [1]:
import numpy as np
import pandas as pd
import math
from numpy.linalg import inv
#hello this is khai 

In [2]:
# Read the Excel file
#basecase = pd.ExcelFile('C:\\Users\\jingy\\Documents\\EE454Lab\\system_basecase.xlsx')
# khai's file location
basecase = pd.ExcelFile("C:\\Users\\kayla\\OneDrive - UW\\EE 454\\myrepo\\EE454Project\\system_basecase.xlsx")

# Read the busdata sheet
busdata = pd.read_excel(basecase, 'BusData')
# Read the linedata sheet
linedata = pd.read_excel(basecase, 'LineData')

# Convert to numpy arrays for later use
busdata_np = np.array(busdata)
linedata_np = np.array(linedata)

# # print data to test if it reads correctly
print("Bus Data:", busdata_np.shape)
pd.DataFrame(busdata_np)


num_buses = len(busdata_np)

Bus Data: (12, 6)


In [3]:
print("Line Data:", linedata_np.shape)
pd.DataFrame(linedata_np)

Line Data: (17, 6)


,0,1,2,3,4,5
0,1.0,2.0,0.01938,0.05917,0.0528,95.0
1,1.0,5.0,0.05203,0.20304,0.0492,100.0
2,2.0,3.0,0.04699,0.19797,0.0438,9999.0
3,2.0,4.0,0.05811,0.17632,0.0340,9999.0
4,2.0,5.0,0.05695,0.07388,0.0346,9999.0
5,3.0,4.0,0.67010,0.17103,0.0128,9999.0
6,4.0,5.0,0.01335,0.04211,0.0391,9999.0
7,4.0,7.0,0.00000,0.55618,0.0000,9999.0
8,5.0,6.0,0.00000,0.25202,0.0000,9999.0
9,6.0,9.0,0.09498,0.19890,0.0176,9999.0


In [4]:
def create_Y_matrix(busdata, linedata):

    # Initialize the admittance matrix Y with all zeros
    Y = np.zeros((num_buses, num_buses), dtype=complex)

    # Process each line
    for line in linedata:
        # Get the bus number also take account the 0 index based numbering
        from_bus = int(line[0]) - 1
        to_bus = int(line[1]) - 1
        R = line[2]
        X = line[3]
        # Get the half-line shunt susceptance
        B_shunt = line[4] / 2

        Z = complex(R, X)
        # Get the line admittance
        Y_line = 1 / Z

        # Add each admittance component to the matrix
        Y[from_bus, from_bus] += Y_line + complex(0, B_shunt)
        Y[to_bus, to_bus] += Y_line + complex(0, B_shunt)
        Y[from_bus, to_bus] -= Y_line
        Y[to_bus, from_bus] -= Y_line

    # (commented out as optional) 
    # Get the conductance (G) and susceptance (B) seperately
    G = np.real(Y)
    B = np.imag(Y)

    return Y

In [5]:
# Get the admittence matrix
Y = create_Y_matrix(busdata_np, linedata_np)
G = np.real(Y)
B = np.imag(Y)
print("Admittance Matrix:\n")

pd.DataFrame(Y)


Admittance Matrix:



,0,1,2,3,4,5,6,7,8,9,10,11
0,6.183452-19.833736j,-4.999132+15.263087j,0.000000+0.000000j,0.000000+0.000000j,-1.1843210+4.6216500j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
1,-4.999132+15.263087j,14.364990-33.568623j,-1.135019+4.781863j,-1.686033+5.115838j,-6.5448060+8.4904350j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
2,0.000000+0.000000j,-1.1350190+4.7818630j,2.536066-5.111153j,-1.401047+0.357590j,0.0000000+0.0000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
3,0.000000+0.000000j,-1.6860330+5.1158380j,-1.401047+0.357590j,9.928061-28.807011j,-6.840981+21.578554j,0.000000+0.000000j,0.000000+1.797979j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
4,-1.184321+4.621650j,-6.5448060+8.4904350j,0.000000+0.000000j,-6.840981+21.578554j,14.570107-38.597128j,0.000000+3.967939j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
5,0.000000+0.000000j,0.0000000+0.0000000j,0.000000+0.000000j,0.000000+0.000000j,0.0000000+3.9679390j,6.579923-17.303083j,0.000000+0.000000j,0.000000+0.000000j,-1.955029+4.094074j,-1.525967+3.175964j,-3.098927+6.102755j,0.000000+0.000000j
6,0.000000+0.000000j,0.0000000+0.0000000j,0.000000+0.000000j,0.000000+1.797979j,0.0000000+0.0000000j,0.000000+0.000000j,5.326055-15.155774j,-3.902050+10.365394j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,-1.424005+3.029050j
7,0.000000+0.000000j,0.0000000+0.0000000j,0.000000+0.000000j,0.000000+0.000000j,0.0000000+0.0000000j,0.000000+0.000000j,-3.902050+10.365394j,5.782934-14.734838j,-1.880885+4.402944j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
8,0.000000+0.000000j,0.0000000+0.0000000j,0.000000+0.000000j,0.000000+0.000000j,0.0000000+0.0000000j,-1.955029+4.094074j,0.000000+0.000000j,-1.880885+4.402944j,3.835913-8.475218j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
9,0.000000+0.000000j,0.0000000+0.0000000j,0.000000+0.000000j,0.000000+0.000000j,0.0000000+0.0000000j,-1.525967+3.175964j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,4.014992-5.405089j,-2.489025+2.251975j,0.000000+0.000000j


In [6]:
V_mag = busdata_np[:, 5]
V_ang = np.zeros(num_buses)
busnum = busdata_np[:,0]
P_Load = busdata_np[:, 1]
Q_Load = busdata_np[:, 2]
P_Gen = busdata_np[:, 4]
bus_type = busdata_np[:, 3]

slack = None
PQind = []
PVind = []

for i in range(num_buses):


    # initialize V based on bus type
    if bus_type[i] == 'S':
        slack = i
    elif bus_type[i] == 'G':
        PVind.append(i)
    elif bus_type[i] == 'D':
        PQind.append(i)

In [7]:
print("Vmag:" + str(V_mag))
print("Vang:" + str(V_ang))

Vmag:[1.04 1.0 1.025 1.0 1.0 1.0 1.0 1.0 1.02 1.02 1.0 1.03]
Vang:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
#formulas in textbook are written as 1 being Slack, 2 to m being PV (gen), and m+1 to N being PQ 

def Pmismatches (vpu, G, B, theta, pgen, pload):
    
    #initialize array
    Pmis = np.zeros(num_buses)
    
    #for each bus...
    for k in range(num_buses):
        
        #initialize a sum for all buses
        sum = 0
        for i in range(num_buses):
            sum +=  vpu[i]*vpu[k]*(G[k, i]*math.cos(math.radians(theta[k]-theta[i]))+B[k, i]*
                                   math.sin(math.radians(theta[k]-theta[i])))
            
        #add sum to load-gen (divided by 100 M for per-unit value) for each bus
        Pmis[k] = sum + pload[k]/100 - pgen[k]/100
        
    return Pmis


def Qmismatches (vpu, G, B, theta, qload):
    #initialize array

    Qmis = np.zeros(num_buses)
    
    #for each bus...
    for k in range(num_buses):
        sum = 0
        
        #initialize a sum for all buses

        for i in range(num_buses):
            sum +=  vpu[i]*vpu[k]*(G[k, i]*math.sin(math.radians(theta[k]-theta[i]))-B[k, i]*
                                   math.cos(math.radians(theta[k]-theta[i])))
        
        #add sum to load (divided by 100 M for per-unit value) for each bus

        Qmis[k] = sum + qload[k]/100
        
    return Qmis


    

In [9]:
print(Pmismatches(V_mag, G, B, V_ang, P_Gen, P_Load))
print(Qmismatches(V_mag, G, B, V_ang, Q_Load))

[ 0.25723161 -0.02734074  0.09698669  0.57297383  0.02862718  0.06538008
  0.29227984  0.0523823  -0.10874737 -0.13609416  0.05110968 -0.10186511]
[ 0.77204344 -0.68767004  0.2619658   0.05711025 -0.262316   -0.09805077
 -0.01152151 -0.06355887  0.17165845  0.10295681 -0.0994884   0.1789003 ]


In [10]:
#mostly done ? 
def HMatrix (G, B, vpu, theta):
    #HMatrix will iterate P(row) from buses 2 to N and theta(col) from buses 2 to N
    
    #rows and cols are the index numbers of buses to be iterated
    rows = np.arange(1, num_buses, dtype=int)
    cols = np.arange(1, num_buses, dtype=int)
  
    
    #empty H array
    H = np.zeros((len(rows), len(cols)))
    
    #r represents the 
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        matchBus = False
        matchCol = None
        
        for i in range(0, num_buses):
            

            
            
            
            if (i != k and i in cols):
                H[r,c] = vpu[k]*vpu[i]*(G[k, i]*math.sin(math.radians(theta[k]-theta[i]))-B[k, i]*
                                            math.cos(math.radians(theta[k]-theta[i])))
#                 print("H has been printed at " + str(r) + " " + str(c))
            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c
                
                
            if (i != k):
                sum += vpu[k]*vpu[i]*(-1*G[k, i]*math.sin(math.radians(theta[k]-theta[i]))+B[k, i]*
                                            math.cos(math.radians(theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
            
                
        if (matchBus):     
        
            H[r,matchCol] = sum
        
        r += 1
        
    return H

In [11]:
def MMatrix (G, B, vpu, theta):
    #MMatrix will iterate P(row) from buses 2 to N and V(col) for PQ buses
    
    #rows and cols are the index numbers of buses to be iterated
    rows = np.arange(1, num_buses, dtype=int)
    cols = PQind
    
    
    #optional check for index values
#     print("rows" + str(rows))
#     print("cols" + str(cols))
    
    #empty M array
    M = np.zeros((len(rows), len(cols)))
    
    
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        
        matchBus = False
        matchCol = None
        
        #print("k is " + str(k))
        #print("hrow is " + str(hr))
        for i in range(0, num_buses):
            
            
            #print("i is " + str(i) + "and mcol is " + str(c))

            
            
            
            if (i != k and i in cols):
                M[r,c] = vpu[k]*(G[k, i]*math.cos(math.radians(theta[k]-theta[i]))+B[k, i]*
                                            math.sin(math.radians(theta[k]-theta[i])))
#                 print("M has been printed at " + str(r) + " " + str(c))
            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c
                    
                
            if (i != k):
                sum += vpu[i]*(G[k, i]*math.cos(math.radians(theta[k]-theta[i]))+B[k, i]*
                                            math.sin(math.radians(theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
        if (matchBus):     
        
            M[r,matchCol] = 2*G[k, k]*vpu[k] + sum
        
        r += 1
        
    return M

In [12]:
def NMatrix (G, B, vpu, theta):
    #NMatrix will iterate Q(row) for PQ buses and theta(col) for PQ and PV buses
    
    #rows and cols are the index numbers of buses to be iterated
    rows = PQind
    cols = np.arange(1, num_buses, dtype=int)
    
    #optional check for index values
#     print("rows" + str(rows))
#     print("cols" + str(cols))
    
    #empty N array
    N = np.zeros((len(rows), len(cols)))
    
    
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        
        matchBus = False
        matchCol = None
#         print("k is " + str(k))
        #print("hrow is " + str(hr))
        for i in range(0, num_buses):
            
#             print("i is " + str(i) + "and Ncol is " + str(c))

            
            
            
            if (i != k and i in cols):
                N[r,c] = vpu[k]*vpu[i]*(-1*G[k, i]*math.cos(math.radians(theta[k]-theta[i]))-B[k, i]*
                                            math.sin(math.radians(theta[k]-theta[i])))
#                 print("N has been printed at " + str(r) + " " + str(c))
            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c     
            
                
            if (i != k):
                sum += vpu[k]*vpu[i]*(G[k, i]*math.cos(math.radians(theta[k]-theta[i]))+B[k, i]*
                                            math.sin(math.radians(theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
        if (matchBus):     
        
            N[r,matchCol] = sum
        
        r += 1
        
    return N

In [13]:
def LMatrix (G, B, vpu, theta):
    #LMatrix will iterate Q(row) for PQ buses and V(col) for PQ buses
    
    #rows and cols are the index numbers of buses to be iterated
    rows = PQind
    cols = PQind
    
    #optional check for index values
#     print("rows" + str(rows))
#     print("cols" + str(cols))
    
    #empty L array
    L = np.zeros((len(rows), len(cols)))
    
    
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        
        matchBus = False
        matchCol = None
        
        #print("k is " + str(k))
        #print("hrow is " + str(hr))
        for i in range(0, num_buses):
            
            #print("i is " + str(i) + "and lcol is " + str(c))

            
            
            
            if (i != k and i in cols):
                L[r,c] = vpu[k]*(G[k, i]*math.sin(math.radians(theta[k]-theta[i]))-B[k, i]*
                                            math.cos(math.radians(theta[k]-theta[i])))
#                 print("L has been printed at " + str(r) + " " + str(c))

            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c
                    
            if (i != k):
                sum += vpu[i]*(G[k, i]*math.sin(math.radians(theta[k]-theta[i]))-B[k, i]*
                                            math.cos(math.radians(theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
        if (matchBus):     
        
            L[r,matchCol] =  -2*B[k, k]*vpu[k] + sum
        
        r += 1
        
    return L

In [14]:
H = HMatrix(G, B, V_mag, V_ang)

pd.DataFrame(H)


,0,1,2,3,4,5,6,7,8,9,10
0,34.381293,-4.901410,-5.115838,-8.490435,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-4.901410,5.267939,-0.366530,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-5.115838,-0.366530,28.858901,-21.578554,0.000000,-1.797979,0.000000,0.000000,0.000000,0.000000,0.000000
3,-8.490435,0.000000,-21.578554,38.843444,-3.967939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,-3.967939,17.486134,0.000000,0.000000,-4.175956,-3.239483,-6.102755,0.000000
5,0.000000,0.000000,-1.797979,0.000000,0.000000,15.283295,-10.365394,0.000000,0.000000,0.000000,-3.119922
6,0.000000,0.000000,0.000000,0.000000,0.000000,-10.365394,14.856397,-4.491003,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,-4.175956,0.000000,-4.491003,8.666958,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,-3.239483,0.000000,0.000000,0.000000,5.536497,-2.297014,0.000000
9,0.000000,0.000000,0.000000,0.000000,-6.102755,0.000000,0.000000,0.000000,-2.297014,10.784182,-2.384412


In [15]:
M = MMatrix(G, B, V_mag, V_ang)

pd.DataFrame(M)


,0,1,2,3,4,5,6
0,14.136649,-1.686033,-6.544806,0.000000,0.000000,0.000000,0.000000
1,-1.163395,-1.436073,0.000000,0.000000,0.000000,0.000000,0.000000
2,-1.686033,9.893034,-6.840981,0.000000,0.000000,0.000000,0.000000
3,-6.544806,-6.840981,14.522734,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,6.510303,0.000000,0.000000,-3.098927
5,0.000000,0.000000,0.000000,0.000000,5.283335,-3.902050,0.000000
6,0.000000,0.000000,0.000000,0.000000,-3.902050,5.745317,0.000000
7,0.000000,0.000000,0.000000,-1.994129,0.000000,-1.918502,0.000000
8,0.000000,0.000000,0.000000,-1.556487,0.000000,0.000000,-2.538805
9,0.000000,0.000000,0.000000,-3.098927,0.000000,0.000000,6.641056


In [16]:
N = NMatrix(G, B, V_mag, V_ang)
print(PQind)
pd.DataFrame(N)


[1, 3, 4, 5, 6, 7, 10]


,0,1,2,3,4,5,6,7,8,9,10
0,-14.593330,1.163395,1.686033,6.544806,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
1,1.686033,1.436073,-9.963087,6.840981,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
2,6.544806,-0.000000,6.840981,-14.617480,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
3,-0.000000,-0.000000,-0.000000,-0.000000,-6.649543,-0.000000,-0.000000,1.994129,1.556487,3.098927,-0.000000
4,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-5.368775,3.902050,-0.000000,-0.000000,-0.000000,1.466726
5,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,3.902050,-5.820552,1.918502,-0.000000,-0.000000,-0.000000
6,-0.000000,-0.000000,-0.000000,-0.000000,3.098927,-0.000000,-0.000000,-0.000000,2.538805,-6.808836,1.171104


In [17]:
L = LMatrix(G, B, V_mag, V_ang)

pd.DataFrame(L)

,0,1,2,3,4,5,6
0,32.755953,-5.115838,-8.490435,0.000000,0.000000,0.000000,0.000000
1,-5.115838,28.755122,-21.578554,0.000000,-1.797979,0.000000,0.000000
2,-8.490435,-21.578554,38.350812,-3.967939,0.000000,0.000000,0.000000
3,0.000000,0.000000,-3.967939,17.120032,0.000000,0.000000,-6.102755
4,0.000000,-1.797979,0.000000,0.000000,15.028252,-10.365394,0.000000
5,0.000000,0.000000,0.000000,0.000000,-10.365394,14.613279,0.000000
6,0.000000,0.000000,0.000000,-6.102755,0.000000,0.000000,10.469205


In [18]:
def build_jacobian(G, B, vpu, theta):
    mat1 = np.concatenate((HMatrix(G, B, vpu, theta),NMatrix(G, B, vpu, theta)), axis=0)
    mat2 = np.concatenate((MMatrix(G, B, vpu, theta),LMatrix(G, B, vpu, theta)), axis=0)

    J = np.concatenate((mat1, mat2), axis=1)
    return J

In [19]:
J = build_jacobian(G, B, V_mag, V_ang)
pd.DataFrame(J)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,34.381293,-4.901410,-5.115838,-8.490435,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.136649,-1.686033,-6.544806,0.000000,0.000000,0.000000,0.000000
1,-4.901410,5.267939,-0.366530,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.163395,-1.436073,0.000000,0.000000,0.000000,0.000000,0.000000
2,-5.115838,-0.366530,28.858901,-21.578554,0.000000,-1.797979,0.000000,0.000000,0.000000,0.000000,0.000000,-1.686033,9.893034,-6.840981,0.000000,0.000000,0.000000,0.000000
3,-8.490435,0.000000,-21.578554,38.843444,-3.967939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.544806,-6.840981,14.522734,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,-3.967939,17.486134,0.000000,0.000000,-4.175956,-3.239483,-6.102755,0.000000,0.000000,0.000000,0.000000,6.510303,0.000000,0.000000,-3.098927
5,0.000000,0.000000,-1.797979,0.000000,0.000000,15.283295,-10.365394,0.000000,0.000000,0.000000,-3.119922,0.000000,0.000000,0.000000,0.000000,5.283335,-3.902050,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,-10.365394,14.856397,-4.491003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.902050,5.745317,0.000000
7,0.000000,0.000000,0.000000,0.000000,-4.175956,0.000000,-4.491003,8.666958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.994129,0.000000,-1.918502,0.000000
8,0.000000,0.000000,0.000000,0.000000,-3.239483,0.000000,0.000000,0.000000,5.536497,-2.297014,0.000000,0.000000,0.000000,0.000000,-1.556487,0.000000,0.000000,-2.538805
9,0.000000,0.000000,0.000000,0.000000,-6.102755,0.000000,0.000000,0.000000,-2.297014,10.784182,-2.384412,0.000000,0.000000,0.000000,-3.098927,0.000000,0.000000,6.641056


In [25]:
def NR (vpu, G, B, theta, pgen, pload, qload):
    
    twoToN =  np.arange(1, num_buses, dtype=int)
    
    allPmis = Pmismatches(vpu, G, B, theta, pgen, pload)
    
    allQmis =  Qmismatches(vpu, G, B, theta, qload)
    
    newTheta = np.copy(theta)
    newV = np.copy(vpu)
    
    
#     firstPMismatch = allPmis[twoToN]
#     firstQMismatch = allQmis[PQind]
    
    
    
    currentPMismatch = allPmis
    currentQMismatch = allQmis
    
    print(currentPMismatch)
    print(currentQMismatch)

    notSmall = True

    while(notSmall):
        
        J = build_jacobian(G, B, newV, newTheta)

        delta = np.matmul(inv(J), np.concatenate((currentPMismatch[twoToN], currentQMismatch[PQind])))

    #     print(delta)
    #     print(delta[0:len(twoToN)])
    #     print(delta[len(twoToN):])

        newTheta[twoToN] = newTheta[twoToN]-delta[0:len(twoToN)]

        newV[PQind] = newV[PQind] - delta[len(twoToN):]

        currentPMismatch = Pmismatches(newV, G, B, newTheta, pgen, pload)
        currentQMismatch = Qmismatches(newV, G, B, newTheta, qload)

        print(currentPMismatch)
        print(currentQMismatch)

        if(np.all(np.abs(currentPMismatch) <= 0.1) and np.all(np.abs(currentQMismatch) <= 0.1)):
            notSmall = False

    
    return newV, newTheta
        

In [26]:
newV, newTheta = NR(V_mag, G, B, V_ang, P_Gen, P_Load, Q_Load)

[ 0.25723161 -0.02734074  0.09698669  0.57297383  0.02862718  0.06538008
  0.29227984  0.0523823  -0.10874737 -0.13609416  0.05110968 -0.10186511]
[ 0.77204344 -0.68767004  0.2619658   0.05711025 -0.262316   -0.09805077
 -0.01152151 -0.06355887  0.17165845  0.10295681 -0.0994884   0.1789003 ]
[ 0.12241414  0.22871867  0.05737491  0.47940563  0.03526241  0.10583914
  0.26572025  0.06290173 -0.12366979 -0.16843361  0.06996559 -0.10129308]
[ 0.28961942  0.00959977  0.13414716 -0.20693977 -0.07611312 -0.02989421
 -0.08436649 -0.03104353  0.1356646   0.05017729 -0.07878719  0.17630728]
[ 0.14287063  0.21856737  0.05795542  0.47130223  0.03456525  0.10309635
  0.26072085  0.06177013 -0.12093016 -0.16491603  0.06858145 -0.09917543]
[ 0.29952596 -0.00821815  0.13775904 -0.20120844 -0.07912164 -0.02965331
 -0.08310891 -0.03052179  0.13605816  0.04887115 -0.07743753  0.17623234]
[ 0.15875967  0.21462323  0.05708447  0.46293648  0.03381819  0.1012413
  0.25610821  0.06065237 -0.1187338  -0.161988

[ 0.9036584   0.03648885  0.01115076  0.08006318  0.00463147  0.0172299
  0.04437233  0.01029681 -0.02008849 -0.0279331   0.01155499 -0.01654159]
[ 0.11332476 -0.00656131  0.13422771 -0.03664842 -0.01784658 -0.00463455
 -0.01498285 -0.00593703  0.10245225 -0.04804816 -0.01354589  0.14409719]
[ 0.90644362  0.03584778  0.01096052  0.07866302  0.00454568  0.01692738
  0.04359644  0.01011596 -0.01973539 -0.02744407  0.01135249 -0.01625109]
[ 0.11262057 -0.00646573  0.1342043  -0.0360165  -0.01755066 -0.00455207
 -0.0147239  -0.0058358   0.10232583 -0.04839367 -0.01331028  0.14397933]
[ 0.90918045  0.03521804  0.01077345  0.07728741  0.00446156  0.01663021
  0.04283414  0.00993831 -0.01938853 -0.02696363  0.01115356 -0.01596573]
[ 0.11192844 -0.00637118  0.13418122 -0.03539535 -0.01725936 -0.00447108
 -0.01446939 -0.00573626  0.10220157 -0.04873311 -0.01307876  0.14386351]
[ 0.91186971  0.03459944  0.0105895   0.0759359   0.00437908  0.01633828
  0.0420852   0.00976379 -0.01904782 -0.026491

[ 1.01689142  0.01058717  0.00330412  0.02331059  0.0012902   0.00500182
  0.0129199   0.00298844 -0.00582696 -0.00812593  0.0033591  -0.00480132]
[ 0.08455442 -0.00214232  0.13321201 -0.01077886 -0.0053915  -0.0013321
 -0.00439931 -0.00175972  0.09729081 -0.06204336 -0.00395916  0.13930215]
[ 1.0177061   0.01040202  0.00324683  0.02290352  0.00126725  0.00491437
  0.01269429  0.00293619 -0.00572505 -0.00798398  0.0033004  -0.00471737]
[ 0.08434636 -0.00210656  0.13320422 -0.0105914  -0.00529874 -0.00130872
 -0.00432275 -0.00172921  0.09725351 -0.06214367 -0.00389012  0.13926762]
[ 1.01850657  0.01022011  0.00319053  0.02250356  0.00124472  0.00482845
  0.01247262  0.00288485 -0.00562492 -0.00784451  0.00324273 -0.00463489]
[ 0.08414192 -0.00207136  0.13319657 -0.01040719 -0.00520755 -0.00128575
 -0.00424751 -0.00169922  0.09721687 -0.06224223 -0.00382229  0.1392337 ]
[ 1.0192931   0.0100414   0.00313519  0.0221106   0.0012226   0.00474404
  0.01225482  0.00283441 -0.00552655 -0.007707

[ 1.05401893e+00  2.16703313e-03  6.80982434e-04  4.77697870e-03
  2.60430568e-04  1.02399569e-03  2.64766262e-03  6.11763450e-04
 -1.19261049e-03 -1.66472589e-03  6.88012983e-04 -9.82908292e-04]
[ 0.07505721 -0.0004547   0.13285028 -0.0022162  -0.00111805 -0.00027183
 -0.00090402 -0.00036269  0.0955887  -0.06660942 -0.00081234  0.13772832]
[ 1.05418614e+00  2.12919568e-03  6.69112923e-04  4.69359564e-03
  2.55867096e-04  1.00611713e-03  2.60144723e-03  6.01082116e-04
 -1.17178662e-03 -1.63566554e-03  6.76001987e-04 -9.65746942e-04]
[ 0.07501437 -0.00044683  0.13284862 -0.00217755 -0.00109859 -0.00026708
 -0.00088825 -0.00035637  0.09558103 -0.06662996 -0.00079817  0.13772123]
[ 1.05435042e+00  2.09201916e-03  6.57450025e-04  4.61166821e-03
  2.51383896e-04  9.88550837e-04  2.55603862e-03  5.90587347e-04
 -1.15132650e-03 -1.60711257e-03  6.64200722e-04 -9.48885334e-04]
[ 0.07497227 -0.0004391   0.13284699 -0.00213957 -0.00107947 -0.00026241
 -0.00087276 -0.00035016  0.09557348 -0.06665

[ 1.06060258e+00  6.77722350e-04  2.13232319e-04  1.49427979e-03
  8.12440482e-05  3.20257860e-04  8.28212070e-04  1.91328600e-04
 -3.72975525e-04 -5.20712142e-04  2.15196234e-04 -3.07405636e-04]
[ 7.33697846e-02 -1.43104744e-04  1.32784654e-01 -6.93651260e-04
 -3.50466146e-04 -8.49660044e-05 -2.82923227e-04 -1.13567241e-04
  9.52863678e-02 -6.74179377e-02 -2.54163198e-04  1.37449155e-01]
[ 1.06065490e+00  6.65892337e-04  2.09512267e-04  1.46819880e-03
  7.98242988e-05  3.14667674e-04  8.13756558e-04  1.87988890e-04
 -3.66465004e-04 -5.11623472e-04  2.11440067e-04 -3.02039773e-04]
[ 7.33563717e-02 -1.40613814e-04  1.32784131e-01 -6.81547523e-04
 -3.44354866e-04 -8.34825172e-05 -2.77986192e-04 -1.11585947e-04
  9.52839648e-02 -6.74243611e-02 -2.49727495e-04  1.37446937e-01]
[ 1.06070630e+00  6.54268848e-04  2.05857089e-04  1.44257305e-03
  7.84293899e-05  3.09175076e-04  7.99553358e-04  1.84707482e-04
 -3.60068143e-04 -5.02693447e-04  2.07749468e-04 -2.96767583e-04]
[ 7.33431928e-02 -1.

...

[ 1.06360020e+00 -4.99600361e-16 -1.11022302e-15  4.10782519e-15
  1.59594560e-15  6.66133815e-16 -8.32667268e-16  1.38777878e-16
 -1.94289029e-16 -5.55111512e-17 -8.88178420e-16  5.55111512e-17]
[ 7.26011464e-02  1.77635684e-15  1.32754623e-01  2.23432384e-15
  1.51129109e-14 -1.73472348e-15 -2.33146835e-15  1.66533454e-16
  9.51486597e-02 -6.77859502e-02  1.66533454e-16  1.37322037e-01]
[ 1.06360020e+00 -1.38777878e-15  6.66133815e-16  2.22044605e-15
 -3.78863607e-15 -2.22044605e-16  4.99600361e-16 -3.05311332e-16
 -1.94289029e-16 -5.55111512e-17  0.00000000e+00  5.55111512e-17]
[ 7.26011464e-02 -3.55271368e-15  1.32754623e-01  1.38777878e-17
 -1.76247905e-15 -8.46545056e-16  1.22124533e-15  1.66533454e-16
  9.51486597e-02 -6.77859502e-02  1.66533454e-16  1.37322037e-01]
[ 1.06360020e+00  2.16493490e-15  0.00000000e+00  1.33226763e-15
 -3.77475828e-15 -2.22044605e-16 -3.88578059e-16  1.38777878e-16
  2.49800181e-16 -5.55111512e-17  0.00000000e+00  5.55111512e-17]
[ 7.26011464e-02  3.

[ 1.06360020e+00 -4.99600361e-16 -1.11022302e-15  4.10782519e-15
  1.59594560e-15  6.66133815e-16 -8.32667268e-16  1.38777878e-16
 -1.94289029e-16 -5.55111512e-17 -8.88178420e-16  5.55111512e-17]
[ 7.26011464e-02  1.77635684e-15  1.32754623e-01  2.23432384e-15
  1.51129109e-14 -1.73472348e-15 -2.33146835e-15  1.66533454e-16
  9.51486597e-02 -6.77859502e-02  1.66533454e-16  1.37322037e-01]
[ 1.06360020e+00 -1.38777878e-15  6.66133815e-16  2.22044605e-15
 -3.78863607e-15 -2.22044605e-16  4.99600361e-16 -3.05311332e-16
 -1.94289029e-16 -5.55111512e-17  0.00000000e+00  5.55111512e-17]
[ 7.26011464e-02 -3.55271368e-15  1.32754623e-01  1.38777878e-17
 -1.76247905e-15 -8.46545056e-16  1.22124533e-15  1.66533454e-16
  9.51486597e-02 -6.77859502e-02  1.66533454e-16  1.37322037e-01]
[ 1.06360020e+00  2.16493490e-15  0.00000000e+00  1.33226763e-15
 -3.77475828e-15 -2.22044605e-16 -3.88578059e-16  1.38777878e-16
  2.49800181e-16 -5.55111512e-17  0.00000000e+00  5.55111512e-17]
[ 7.26011464e-02  3.

[ 1.06360020e+00 -4.99600361e-16 -1.11022302e-15  4.10782519e-15
  1.59594560e-15  6.66133815e-16 -8.32667268e-16  1.38777878e-16
 -1.94289029e-16 -5.55111512e-17 -8.88178420e-16  5.55111512e-17]
[ 7.26011464e-02  1.77635684e-15  1.32754623e-01  2.23432384e-15
  1.51129109e-14 -1.73472348e-15 -2.33146835e-15  1.66533454e-16
  9.51486597e-02 -6.77859502e-02  1.66533454e-16  1.37322037e-01]
[ 1.06360020e+00 -1.38777878e-15  6.66133815e-16  2.22044605e-15
 -3.78863607e-15 -2.22044605e-16  4.99600361e-16 -3.05311332e-16
 -1.94289029e-16 -5.55111512e-17  0.00000000e+00  5.55111512e-17]
[ 7.26011464e-02 -3.55271368e-15  1.32754623e-01  1.38777878e-17
 -1.76247905e-15 -8.46545056e-16  1.22124533e-15  1.66533454e-16
  9.51486597e-02 -6.77859502e-02  1.66533454e-16  1.37322037e-01]
[ 1.06360020e+00  2.16493490e-15  0.00000000e+00  1.33226763e-15
 -3.77475828e-15 -2.22044605e-16 -3.88578059e-16  1.38777878e-16
  2.49800181e-16 -5.55111512e-17  0.00000000e+00  5.55111512e-17]
[ 7.26011464e-02  3.

[ 1.06360020e+00 -1.38777878e-15  6.66133815e-16  2.22044605e-15
 -3.78863607e-15 -2.22044605e-16  4.99600361e-16 -3.05311332e-16
 -1.94289029e-16 -5.55111512e-17  0.00000000e+00  5.55111512e-17]
[ 7.26011464e-02 -3.55271368e-15  1.32754623e-01  1.38777878e-17
 -1.76247905e-15 -8.46545056e-16  1.22124533e-15  1.66533454e-16
  9.51486597e-02 -6.77859502e-02  1.66533454e-16  1.37322037e-01]
[ 1.06360020e+00  2.16493490e-15  0.00000000e+00  1.33226763e-15
 -3.77475828e-15 -2.22044605e-16 -3.88578059e-16  1.38777878e-16
  2.49800181e-16 -5.55111512e-17  0.00000000e+00  5.55111512e-17]
[ 7.26011464e-02  3.55271368e-15  1.32754623e-01 -3.76088050e-15
 -9.75608483e-15 -8.46545056e-16  1.22124533e-15 -7.21644966e-16
  9.51486597e-02 -6.77859502e-02  1.66533454e-16  1.37322037e-01]
[ 1.06360020e+00 -4.99600361e-16 -1.11022302e-15  4.10782519e-15
  1.59594560e-15  6.66133815e-16 -8.32667268e-16  1.38777878e-16
 -1.94289029e-16 -5.55111512e-17 -8.88178420e-16  5.55111512e-17]
[ 7.26011464e-02  1.

KeyboardInterrupt: 

In [ ]:
a = np.arange(3)
print(a)
print(np.any(a>8))

ab = ([True, True, False])
a[ab] = [10,10]
print(a.shape)